In [52]:
# imports
import os
from os.path import join
import numpy as np
import pandas as pd
import json
import librosa
from pathlib import Path
import yaml
from tqdm import tqdm
import pickle
from datasets import Dataset, DatasetDict

In [104]:
# load the pickle data file
with open('./pkl/magister_data_flac_16000_train.pkl', 'rb') as f:
    df_train = pickle.load(f)

with open('./pkl/magister_data_flac_16000_dev.pkl', 'rb') as f:
    df_dev = pickle.load(f)

In [105]:
df_train

,file,audio,text
0,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [-0.00091552734, -0.0007324219, -0.0...",FCS GUNNERY BROADCAST POLICY SURFACE AND AIR F...
1,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [-0.00012207031, 0.00018310547, 0.00...",ROGER
2,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [0.0077209473, 0.009490967, 0.011291...",AGUN ENGAGE MISSILE NORTH AT TWO MILES
3,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [-0.0057373047, -0.0053100586, -0.00...",G P M G PROVIDE ONE BOX TWO FIVE ZERO ROUNDS L...
4,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [3.0517578e-05, -3.0517578e-05, -6.1...",OOW COME LEFT ZERO TWO ZERO INCREASE ONE TWO OOW
...,...,...,...
1813,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [0.008239746, 0.011108398, 0.0139770...",TPS PWO CHECK COPY
1814,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [0.0015563965, 0.0008544922, 0.00015...",AND FIRE ON PAC TARGET ZERO ZERO FOUR NINE K A...
1815,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [0.2989502, 0.3025818, 0.3062439, 0....",# OW PWO COME LEFT TWO NINE ZERO WEPS THIS PWO...
1816,./datasets/magister_data_flac_16000_finetune/t...,"{'array': [3.0517578e-05, -0.00024414062, -0.0...",# ALL STATION # THIS PWO MISSIONS BRIEF AS FOL...


In [106]:
df_dev.head()

,file,audio,text
0,./datasets/magister_data_flac_16000_finetune/d...,"{'array': [0.01727295, 0.01184082, 0.006408691...",FOXTROT ONE DELTA THIS IS ALFA FOUR QUEBEC SHO...
1,./datasets/magister_data_flac_16000_finetune/d...,"{'array': [0.0015258789, 0.001159668, 0.000793...",FOXTROT ONE DELTA THIS IS ALFA FOUR QUEBEC SHO...
2,./datasets/magister_data_flac_16000_finetune/d...,"{'array': [0.005645752, 0.004852295, 0.0040588...",ALFA FOUR QUEBEC # POSITIONING # FOR P A C RUN...
3,./datasets/magister_data_flac_16000_finetune/d...,"{'array': [0.001739502, 0.0017700195, 0.001831...",ALL IN THIS IS PWO INTEND CONTINUES FIRING NO ...
4,./datasets/magister_data_flac_16000_finetune/d...,"{'array': [0.0011291504, 0.0005493164, 0.0, 0....",FOXTROT ONE DELTA THIS IS ALFA FOUR QUEBEC COM...


In [107]:
# from now on load the data from the pickle file and not from the dataset class from huggingface

In [108]:
df_dev['file'][0]

'./datasets/magister_data_flac_16000_finetune/dev/11251/314114000/11251-314114000-0045.flac'

In [109]:
df_dev['audio'][0]['array'][:25]

array([ 0.01727295,  0.01184082,  0.00640869,  0.00616455,  0.00592041,
        0.00735474,  0.00881958,  0.00918579,  0.009552  ,  0.00881958,
        0.00811768,  0.00979614,  0.01147461,  0.01074219,  0.01000977,
        0.0083313 ,  0.00668335,  0.0062561 ,  0.00582886,  0.00094604,
       -0.00390625, -0.00979614, -0.01565552, -0.01123047, -0.0067749 ],
      dtype=float32)

In [110]:
# extract all characters available in the annotations
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return vocab

In [111]:
# extract characters from train dataset
vocabs_train = extract_all_chars(df_train)
print(len(vocabs_train))
vocabs_train

36


['N',
 'H',
 '5',
 'O',
 'E',
 'P',
 'C',
 'D',
 'B',
 'A',
 "'",
 'R',
 'X',
 'S',
 'F',
 'L',
 'G',
 ' ',
 'W',
 '4',
 'T',
 'Y',
 'K',
 'Z',
 'Q',
 '8',
 'M',
 'V',
 '2',
 '9',
 '1',
 '0',
 'I',
 'U',
 '#',
 'J']

In [112]:
# extract characters from train dataset
vocabs_dev = extract_all_chars(df_dev)
print(len(vocabs_dev))
vocabs_dev

30


['N',
 'H',
 'O',
 'E',
 'P',
 'C',
 'D',
 'B',
 'A',
 "'",
 'R',
 'X',
 'S',
 'F',
 'L',
 'G',
 ' ',
 'W',
 'T',
 'Y',
 'K',
 'Z',
 'Q',
 'M',
 'V',
 '2',
 'I',
 'U',
 '#',
 'J']